In [1]:
import os
import sys

src_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
util_path = os.path.abspath(os.path.join(src_path, f'Util'))

print(util_path)
if src_path not in sys.path or util_path not in sys.path:
    sys.path.append(src_path)
    sys.path.append(util_path)
DATA_BASEPATH = os.path.abspath(os.path.join(os.getcwd(), "..","../../Data"))

/sfs/gpfs/tardis/home/ujx4ab/ondemand/CBM_Final_Project/src/Util


In [2]:
from pathlib import Path
import json
import numpy as np  
import copy

from Util.data_utils import load_reddit_posts, get_comments_for_post, get_user_comment_history, get_comments_in_post_thread, upload_json_to_file
from LLM.Mistral.minstral_utils import generate_mistral_prompt_zero_shot
from Util.posttree import PostTree, UserCommentHistories
from minstral_utils import load_quantized_model, initialize_tokenizer


/home/ujx4ab/ondemand/CBM_Final_Project/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.3'
model = load_quantized_model(model_name)
tokenizer = initialize_tokenizer(model_name)

def run_prompt_on_mistral(prompt):
    stop_token_ids = [0]
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded[0].split("[/INST]", 1)[1].strip()
        

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:17<00:00,  5.76s/it]


In [4]:
POST_PATH = os.path.abspath(os.path.join(DATA_BASEPATH, 'arcticshift/processed/r_books_posts.csv'))
POST_ID = '18vuw2v'
COMMENTPATH = os.path.abspath(os.path.join(DATA_BASEPATH, 'arcticshift/cleaned/r_books_comments_cleaned_and_pruned.csv'))
GEN_METHOD = 'zero_shot' # zero_shot, few_shot, instruct
POST_FOLDER_PATH = os.path.abspath(os.path.join(DATA_BASEPATH, f'eval_results/{GEN_METHOD}/{POST_ID}'))
USER_HIST_PATH = os.path.abspath(os.path.join(DATA_BASEPATH, f'arcticshift/cleaned/r_books_comments_cleaned.csv'))

In [5]:
post_tree = PostTree(
    POST_PATH,
    POST_ID
    )
post_tree.create_comment_tree(COMMENTPATH)
os.makedirs(POST_FOLDER_PATH, exist_ok=True)
prompt_response_file = os.path.join(POST_FOLDER_PATH, "prompt_response.txt")

new_tree = copy.deepcopy(post_tree)

user_histories = UserCommentHistories(
    USER_HIST_PATH,
    POST_ID
    )

list_lengths = [len(lst) for lst in user_histories.user_histories.values()]
mean_length = np.mean(list_lengths)
std_length = np.std(list_lengths)
print(f"Mean length: {mean_length}")
print(f"Standard deviation: {std_length}")

Mean length: 10.172043010752688
Standard deviation: 25.24444947229867


In [7]:
for node in post_tree.bfs_generator():
    # get the current comment and 10 of their history comments selected at random (if they have 10)
    cur_user_comment_history = user_histories.get_random_user_history(f"{node.user}")

    # get  previous responses on thread
    previous_comments = node.get_previous_responses()

    # get comment to be responded to and delete from response history
    try: 
        if len(previous_comments) > 0:
            comment_of_interest = previous_comments[-1]
        else:
            comment_of_interest = ""
        previous_comments = "\n".join(previous_comments)
    except TypeError: 
        comment_of_interest = ""
        previous_comments = ""

    prompt = generate_mistral_prompt_zero_shot(
        post_title=post_tree.title, 
        post_content=post_tree.content, 
        previous_comments_on_thread=previous_comments, 
        comment_of_interest=comment_of_interest,
        user_comment_history=cur_user_comment_history, 
    )
    
    response = run_prompt_on_mistral(prompt)
    node.comment_text = response

    with open(prompt_response_file, "a") as file:
        file.write(f"{prompt}\n")
        file.write(f"{response}\n")
    
    #print(response)    

post_tree.save_as_json(f"/home/ujx4ab/ondemand/CBM_Final_Project/Data/posttrees/{post_tree.post_id}2.json")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin